In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"\

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-camp/notebook/03-experiment-tracking/mlruns/1', creation_time=1716229855130, experiment_id='1', last_update_time=1716229855130, lifecycle_stage='active', name='my-first-experiment', tags={}>]

In [8]:
client.create_experiment(name = "my_cool_experiment")

'2'

In [9]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [10]:
for run in runs:
    print(f"run id: {run.info.run_id},rmse: {run.data.metrics['rmse']:4f}")

run id: b7eed26e9ff2495f9b5e92a6deab969a,rmse: 4.965459
run id: 12d40f85651c4413974cbd05fef76db3,rmse: 4.965459
run id: 13273f90f92e4a4ba02a9518d975ac9a,rmse: 4.965459
run id: 20f2ffe1584241d7a76e8e06c54ac3b4,rmse: 4.966782
run id: d139c463f53b4287b793d96a81c36266,rmse: 4.976702


In [11]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [13]:
run_id = '13273f90f92e4a4ba02a9518d975ac9a'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name="first_model_xgboost")

Registered model 'first_model_xgboost' already exists. Creating a new version of this model...
Created version '1' of model 'first_model_xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1716748174771, current_stage='None', description=None, last_updated_timestamp=1716748174771, name='first_model_xgboost', run_id='13273f90f92e4a4ba02a9518d975ac9a', run_link=None, source='/workspaces/mlops-camp/notebook/03-experiment-tracking/mlruns/1/13273f90f92e4a4ba02a9518d975ac9a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [15]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716748158161, description=None, last_updated_timestamp=1716748327372, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716748327372, current_stage='None', description='', last_updated_timestamp=1716748327372, name='first_model_xgboost', run_id='b7eed26e9ff2495f9b5e92a6deab969a', run_link='', source='/workspaces/mlops-camp/notebook/03-experiment-tracking/mlruns/1/b7eed26e9ff2495f9b5e92a6deab969a/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='first_model_xgboost', tags={}>]

In [22]:
model_name = "first_model_xgboost"
latest_versions =client.search_model_versions('name'==model_name)

for version in latest_versions:
    print(f"version:{version.version}, stage:{version.current_stage}")



version:2, stage:None
version:1, stage:None


In [24]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage='Staging',
    archive_existing_versions=True

)

/tmp/ipykernel_17912/1796233148.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716748327372, current_stage='Staging', description='', last_updated_timestamp=1716750321652, name='first_model_xgboost', run_id='b7eed26e9ff2495f9b5e92a6deab969a', run_link='', source='/workspaces/mlops-camp/notebook/03-experiment-tracking/mlruns/1/b7eed26e9ff2495f9b5e92a6deab969a/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [27]:
from datetime import datetime
model_version=2
new_stage ='staging'
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716748327372, current_stage='Staging', description='The model version 2 was transitioned to staging on 2024-05-26', last_updated_timestamp=1716751349270, name='first_model_xgboost', run_id='b7eed26e9ff2495f9b5e92a6deab969a', run_link='', source='/workspaces/mlops-camp/notebook/03-experiment-tracking/mlruns/1/b7eed26e9ff2495f9b5e92a6deab969a/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>